In [34]:
import folium
import pandas as pd
from IPython.display import display
import gpxpy
import gpxpy.gpx
import os
import numpy as np

## High Level View of Vancouver+

In [67]:
route_map = folium.Map(
    location=[49.2578182,-123.2063049],
    zoom_start=11,
    tiles='CartoDBDark_Matter',
    width=2040,
    height=1200
)
display(route_map)

## Filter Activites Around Vancouver and Bike Rides

In [33]:
def list_files(filepath, filetype):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for file in files:
         if file.lower().endswith(filetype.lower()):
            paths.append(os.path.join(root, file))
   return(paths)

gpx_dir = 'C:/Users/adamf/Downloads/export_45952748/activities_gpx/'

files = list_files(gpx_dir, ".gpx")

max_lat = 49.9
min_lat = 48.45
max_lon = -121.16
min_lon = -126.00

in_bounds = []

for file in files:
    gpx_file = open(file, 'r')
    gpx = gpxpy.parse(gpx_file)
    bounds = gpx.tracks[0].segments[0].get_bounds()
    
    # GPX without bounds should not be included
    if bounds is None:
       continue

    if gpx.tracks[0].type != "Ride":
       continue

    if bounds.min_latitude > min_lat and bounds.max_latitude < max_lat and bounds.min_longitude > min_lon and bounds.max_longitude < max_lon:
       in_bounds.append(file)





## Add Suitable Rides to the Map

In [ ]:
# Activities that have a big jump between points because I forgot to unpause :') or that include ferries 
blacklist = [gpx_dir + "3579106692.gpx", gpx_dir + "10226554548.gpx", gpx_dir + "10226533650.gpx", gpx_dir + "5233209498.gpx", gpx_dir + "10515855906.gpx", gpx_dir + "7085179530.gpx", gpx_dir + "10145677460", gpx_dir + "5233209498.gpx", gpx_dir + "6311433741.gpx", gpx_dir + "6277707880.gpx", gpx_dir + "6809940397.gpx", gpx_dir + "6123862131.gpx", gpx_dir + "6211073658.gpx"]

for file in in_bounds:
    if file in blacklist:
        continue
    
    gpx_file = open(file, 'r')
    gpx = gpxpy.parse(gpx_file)

    coords = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coords.append([point.latitude, point.longitude])


    folium.PolyLine(np.array(coords), weight = 3, color = "#f5ad42", tooltip=file, opacity=0.3).add_to(route_map)
display(route_map)

## Sum Total Distances for Rides

In [76]:
# strava says I've ridden 
total_distance = 12376 # km
total_elevation_gain = 161532 # meters
total_time = 871 # hours

## Source: https://ourworldindata.org/travel-carbon-footprint
car_emissions = 170 # g / km
bike_emissions = (50 - 16) / 2 # g / km

## Source: https://curb6.com/footprint/flights/vancouver-yvr/toronto-yyz
van_to_toronto_emissions  = 555.1 # kg CO2

total_car_emissions = total_distance * car_emissions 
total_bike_emissions = total_distance * bike_emissions

emissions_savings = total_car_emissions - total_bike_emissions
print(emissions_savings / 1000, "kg of CO2 Emissions Saved")
print("\nSimilar to taking {f} flights from Vancouver to Toronto".format( f = round(emissions_savings / 1000 / van_to_toronto_emissions)))

1893.528 kg of CO2 Emissions Saved

Similar to taking 3 flights from Vancouver to Toronto
